In [1]:
import modelbuild
import tensorflow as tf
import keras

2024-01-20 12:11:34.626205: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-20 12:11:34.626239: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-20 12:11:34.627342: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-20 12:11:34.633361: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-20 12:11:35.302653: W tensorflow/compiler/tf2

In [2]:
model1 = modelbuild.buildmodel()
model2 = modelbuild.buildmodel()

In [3]:
img_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
data_flow = img_gen.flow_from_directory('./data',target_size=(224,224),class_mode="binary")

Found 24959 images belonging to 2 classes.


In [7]:
model1.compile()
model1.model.evaluate(data_flow)

2024-01-17 16:26:45.210933: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-01-17 16:26:45.308425: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-17 16:26:45.900541: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


 60/780 [=>............................] - ETA: 30s - loss: 0.7004 - binary_accuracy: 0.4891

/home/ailab/anaconda3/envs/tan/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:868: UserWarning: Truncated File Read
  warnings.warn(str(msg))


780/780 [==============================] - 36s 44ms/step - loss: 0.6980 - binary_accuracy: 0.4996


[0.6980067491531372, 0.4995793104171753]

In [8]:
model2.compile()
model2.model.evaluate(data_flow)

780/780 [==============================] - 32s 40ms/step - loss: 0.6980 - binary_accuracy: 0.4996


[0.6980071663856506, 0.4995793104171753]

In [4]:
model1.model.save('model.h5')
model2.model = keras.models.load_model('model.h5')
# model2.model.evaluate(data_flow)

/home/ailab/anaconda3/envs/tan/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [5]:
data = data_flow.next()
x,y =data
y = y.reshape(-1,1)
y.shape

(32, 1)

In [5]:
import time
epochs = 7
start_train_time = time.time() 
for epoch in range(epochs):
    start_epoch = time.time()
    print("\nStart of epoch %d" % (epoch,))
    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(data_flow):
        y_batch_train = y_batch_train.reshape(-1,1)
        # print(step, x_batch_train.shape, y_batch_train.shape)
        loss_value,grads = model1.train_step(x_batch_train,y_batch_train)
        model2.optimize_model(grads)
        # model1.model = model2.model
        if (step %2 != 0):
            model1.model.set_weights(model2.model.get_weights())
        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %s samples" % ((step + 1) * 32))
        if step==len(data_flow)-1:
            # print(model1.model.evaluate(data_flow))
            # print(model2.model.evaluate(data_flow))
            break
    # Display metrics at the end of each epoch.
    # train_acc = model1.metrics.result()
    # print("Training acc over epoch: %.4f" % (float(train_acc),))
    print("Training time over epoch:", time.time()-start_epoch)
    model1.model.evaluate(data_flow)
    model2.model.evaluate(data_flow)
    # Reset training metrics at the end of each epoch
    model1.metrics.reset_states()
print("Total time:", time.time()-start_train_time)



Start of epoch 0


2024-01-19 02:19:28.218217: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-01-19 02:19:28.316854: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-19 02:19:28.865141: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-19 02:19:30.097980: W external/local_tsl/tsl/framework/bfc_allocator.cc:366] Garbage collection: deallocate free memory regions (i.e., allocations) so that we can re-allocate a larger region to avoid OOM due to memory fragmentation. If you see this message frequently, you are running near the threshold of the available device memory and re-allocation may incur great performance overhead. You may try smaller batch sizes to observe the performance impact. Set TF_ENABLE_GPU_GARBAGE_COLLECTION=false if you'd like to disable this feature.
2024-01-19 02:19:32.260209: I externa

Training loss (for one batch) at step 0: 0.7065
Seen so far: 32 samples
Training loss (for one batch) at step 200: 0.6877
Seen so far: 6432 samples
Training loss (for one batch) at step 400: 0.6927
Seen so far: 12832 samples


/home/ailab/anaconda3/envs/tan/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:868: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Training loss (for one batch) at step 600: 0.6903
Seen so far: 19232 samples
Training time over epoch: 81.55457854270935
780/780 [==============================] - 31s 39ms/step - loss: 0.6837 - binary_accuracy: 0.5518

Start of epoch 1
Training loss (for one batch) at step 0: 0.7671
Seen so far: 32 samples
Training loss (for one batch) at step 200: 0.6645
Seen so far: 6432 samples
Training loss (for one batch) at step 400: 0.6926
Seen so far: 12832 samples
Training loss (for one batch) at step 600: 0.7023
Seen so far: 19232 samples
Training time over epoch: 72.83371424674988
780/780 [==============================] - 31s 39ms/step - loss: 0.6828 - binary_accuracy: 0.5582

Start of epoch 2
Training loss (for one batch) at step 0: 0.7249
Seen so far: 32 samples
Training loss (for one batch) at step 200: 0.6841
Seen so far: 6432 samples
Training loss (for one batch) at step 400: 0.6267
Seen so far: 12832 samples
Training loss (for one batch) at step 600: 0.6926
Seen so far: 19232 samples

In [5]:
import time
epochs = 7
start_train_time = time.time() 
for epoch in range(epochs):
    start_epoch = time.time()
    print("\nStart of epoch %d" % (epoch,))
    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(data_flow):
        y_batch_train = y_batch_train.reshape(-1,1)
        # print(step, x_batch_train.shape, y_batch_train.shape)
        loss_value,grads = model1.train_step(x_batch_train,y_batch_train)
        model2.optimize_model(grads)
        # model1.model = model2.model
        # if step %2:
        model1.model.set_weights(model2.model.get_weights())
        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %s samples" % ((step + 1) * 32))
        if step==len(data_flow)-1:
            # print(model1.model.evaluate(data_flow))
            # print(model2.model.evaluate(data_flow))
            break
    # Display metrics at the end of each epoch.
    # train_acc = model1.metrics.result()
    # print("Training acc over epoch: %.4f" % (float(train_acc),))
    print("Training time over epoch:", time.time()-start_epoch)
    model1.model.evaluate(data_flow)
    model2.model.evaluate(data_flow)
    # Reset training metrics at the end of each epoch
    model1.metrics.reset_states()
print("Total time:", time.time()-start_train_time)



Start of epoch 0


2024-01-18 23:46:59.002623: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-01-18 23:46:59.099963: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-18 23:46:59.636345: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-18 23:47:00.887361: W external/local_tsl/tsl/framework/bfc_allocator.cc:366] Garbage collection: deallocate free memory regions (i.e., allocations) so that we can re-allocate a larger region to avoid OOM due to memory fragmentation. If you see this message frequently, you are running near the threshold of the available device memory and re-allocation may incur great performance overhead. You may try smaller batch sizes to observe the performance impact. Set TF_ENABLE_GPU_GARBAGE_COLLECTION=false if you'd like to disable this feature.
2024-01-18 23:47:03.059474: I externa

Training loss (for one batch) at step 0: 0.7048
Seen so far: 32 samples
Training loss (for one batch) at step 200: 0.6919
Seen so far: 6432 samples
Training loss (for one batch) at step 400: 0.7001
Seen so far: 12832 samples
Training loss (for one batch) at step 600: 0.6878
Seen so far: 19232 samples


/home/ailab/anaconda3/envs/tan/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:868: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Training time over epoch: 112.27105522155762
780/780 [==============================] - 31s 39ms/step - loss: 0.6927 - binary_accuracy: 0.5331

Start of epoch 1
Training loss (for one batch) at step 0: 0.6914
Seen so far: 32 samples
Training loss (for one batch) at step 200: 0.7096
Seen so far: 6432 samples
Training loss (for one batch) at step 400: 0.7006
Seen so far: 12832 samples
Training loss (for one batch) at step 600: 0.6925
Seen so far: 19232 samples
Training time over epoch: 98.19841504096985
780/780 [==============================] - 31s 40ms/step - loss: 0.6911 - binary_accuracy: 0.5626

Start of epoch 2
Training loss (for one batch) at step 0: 0.6925
Seen so far: 32 samples
Training loss (for one batch) at step 200: 0.6688
Seen so far: 6432 samples
Training loss (for one batch) at step 400: 0.6747
Seen so far: 12832 samples
Training loss (for one batch) at step 600: 0.6801
Seen so far: 19232 samples
Training time over epoch: 98.0162079334259
780/780 [=======================

In [4]:
model = modelbuild.buildmodel()

In [ ]:
for i in range(1,16):
    model.model = keras.models.load_model('model1'+str(i)+'.h5')
    print(model.model.evaluate(data_flow))

2024-01-20 12:11:49.215634: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-01-20 12:11:49.309249: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-20 12:11:49.851937: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


423/780 [===============>..............] - ETA: 14s - loss: 0.6926 - binary_accuracy: 0.4972

/home/ailab/anaconda3/envs/tan/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:868: UserWarning: Truncated File Read
  warnings.warn(str(msg))


780/780 [==============================] - 34s 42ms/step - loss: 0.6923 - binary_accuracy: 0.5004
[0.6922769546508789, 0.5004206895828247]
780/780 [==============================] - 33s 42ms/step - loss: 0.6855 - binary_accuracy: 0.5615
[0.6854561567306519, 0.5614808201789856]
308/780 [==========>...................] - ETA: 18s - loss: 0.6812 - binary_accuracy: 0.5686

In [5]:
model.model = keras.models.load_model('model.h5')
print(model.model.evaluate(data_flow))

2024-01-19 12:27:46.989800: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-01-19 12:27:47.082496: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-19 12:27:47.619333: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


219/780 [=======>......................] - ETA: 21s - loss: 0.7055 - binary_accuracy: 0.4980

/home/ailab/anaconda3/envs/tan/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:868: UserWarning: Truncated File Read
  warnings.warn(str(msg))


780/780 [==============================] - 32s 39ms/step - loss: 0.7051 - binary_accuracy: 0.4996
[0.7051015496253967, 0.4995793104171753]
